# Project Title
Project Description

#### The notebook is structured as follows:
1. [Import and Prepare the Data](#data): We will read the price and volume data for the stocks belonging to the S&P 500 list. 

2. [Get the Strategy Performance](#performance): We will create a function to use it both in the train and test data.

3. [Backtest the Strategy Over All Stocks](#backtest): We will use the above function to backtest the strategy with the train data.

4. [Choose the Best Five Stocks](#choose): We will choose five stocks with the highest Sharpe ratio. These ratios will be computed in the previous section.

## Import Libraries

In [19]:
# For data manipulation and visualisation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# For technical indicators computation
import talib as ta
import yfinance as yf
import datetime as dt

# For filtering warnings
import warnings
warnings.filterwarnings("ignore")

# For strategy analytics
# import data_modules.quantra_analytics as qa

In [20]:
# Get list of tickers in SP500 from Wikipedia page
tickers = list(pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]["Symbol"].dropna())
for i in range(len(tickers)):
  tickers[i] = tickers[i].replace(".","-")
# Download data from Yahoo Finance 
data = yf.download(tickers,"2012-01-01")
# Remove stocks with missing values
data = data.iloc[1:].dropna(axis = 1)

[*********************100%***********************]  503 of 503 completed


In [ ]:
# Create Function to store a new trade in the trade book
def trade(data, time, entry_time, entry_price):
    exit_price = data.loc[time, 'Close'] 
    # Calculate PnL
    pnl = round(exit_price - entry_price, 2)
    # Calculate trading cost due to fees
    trading_cost = exit_price * 0.0002 * 2
    pnl -= trading_cost
    trades = pd.DataFrame(columns = ['Position', 'Entry Time', 'Entry Price', 'Exit Time', 'Exit Price', 'PnL'])
    return trades.append({
        'Position': 'Long',
        'Entry Time' : entry_time,
        'Entry Price' : entry_price,
        'Exit Time' : time, 
        'Exit Price': exit_price, 
        'PnL' : pnl
    })


# Create the function to backtest the strategy
def performance(stock, period, visualize):
    """
    Function used to backtest the strategy for each stock.
    """
    # Copy the close prices
    data = stock.copy()
    # Create the Bollinger bands
    data['upper'], data['middle'], data['lower'] = ta.BBANDS(data['Close'], timeperiod=20)
    # Create the band's width
    data['bandwidth'] = (data['upper'] - data['lower'])/data['middle']
    # Calculate the rolling width
    data['rolling_bandwidth'] = data['bandwidth'].rolling(period).mean
    # Compute the condition 1
    data['condition1'] = data['rolling_bandwidth'] > data['rolling_bandwidth']
    # Compute the condition 2
    data['condition2'] = data['High'] > data['upper']
    # Compute the signal: First step
    data['long_signal'] = np.where(data['condition1'] & data['condition2'], 1, np.nan)
    # Compute the signal: Second step
    data['long_signal'] = np.where(data['Low'] < data['middle'], 0, data['long_signal'])
    # Fill the NaN values with previous observations
    data['long_signal'] = data['long_signal'].fillna(method = "ffill")
    # Define variables to keep track of current position, take profit and stop loss multiples of ATR 
    current_position = 0
    stop_loss_multiple = 1.5
    take_profit_multiple = 2.5
    # Calculate the Average True Range(ATR)
    data['ATR'] = ta.ATR(data['High'], data['Low'], data['Close'], timeperiod=14)
    # Entry time and price of when we buy the stock
    entry_time = np.nan
    entry_price = np.nan

    # Create the trade book where all the trades get stored
    trade_book = pd.DataFrame(columns = ['Position', 'Entry Time', 'Entry Price', 'Exit Time', 'Exit Price', 'PnL'])

    # Create the loop to backtest
    for time in data.index:
        # Condition to go long
        if current_position == 0 and data.loc[time, 'long_signal'] == 1:
            current_position = 1
            entry_time = time
            entry_price = data.loc[time, 'Close']
        elif current_position == 1:
            # Specify the stop loss target as the max between a multiple of the ATR or the middle Bollinger band
            stop_loss = max(entry_price - data.loc[time, 'ATR'] * stop_loss_multiple, data.loc['time', 'middle'])
            # Specify the take profit target
            take_profit = entry_price + data.loc[time, 'ATR'] * take_profit_multiple
            # Condition to exit the Long position
            if data.loc[time, 'Close'] > take_profit or data.loc[time, 'Close'] < stop_loss:
                trade_book = pd.concat(trade_book, trade(data, time, entry_time, entry_price))
                trade_book = trade_book.reset_index(drop = True)
                current_position = 0
            
    # Create the long_position column
    data['long_position'] = np.nan
    data.loc[data.index.isin(trade_book['Entry Time']), 'long_position'] = 1
    data.loc[data.index.isin(trade_book['Exit Time']), 'long_position'] = 0
    data['long_position'] = data['long_position'].fillna(method = 'ffill')
    # Compute the strategy returns
    data['returns'] = data['long_position'].shift(1) * data['Close'].pct_change()
    # Compute the performance metrics
    sharpe = qa.sharpe_ratio(data['returns'])
    cagr = qa.cagr(data['returns'])
    mdd = qa.mdd(data['returns'])

    # Return the PnL, number of trades, number of positive trades, and the three performance metrics
